In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
print(labels.head())
print(labels.shape)
print(labels['breed'].value_counts())

In [ ]:
#one hot encode labels as needed in the submission format
encoded_labels = pd.get_dummies(labels['breed'])
labels_array = np.asarray(encoded_labels)

In [ ]:
#img parameters
img_rows = 128
img_cols = 128
channels = 3 #coloured image

In [ ]:
x_feature = []
y_feature = []

i=0
for f,img in tqdm(labels.values):
    train_img = cv2.imread('../input/dog-breed-identification/train/{}.jpg'.format(f))
    label = labels_array[i]
    train_img_resize = cv2.resize(train_img,(img_rows,img_cols))
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i+=1

In [ ]:
x_train_data = np.array(x_feature, np.float32) / 255.
x_train_data = x_train_data.reshape(-1,img_rows,img_cols,3)
print(x_train_data.shape)

In [ ]:
y_train_data = np.array(y_feature)
print (y_train_data.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train ,x_val ,y_train ,y_val = train_test_split(x_train_data,y_train_data,test_size=0.25,random_state=2)
print(x_train.shape)
print(x_val.shape)

In [ ]:
submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
test_img = submission['id']
print(test_img.head())

In [ ]:
x_test_feature=[]
for f in test_img.values:
    test_img = cv2.imread('/kaggle/input/dog-breed-identification/test/{}.jpg'.format(f))
    test_img = cv2.resize(test_img,(img_rows,img_cols))
    x_test_feature.append(test_img)

In [ ]:
x_test_data = np.array(x_test_feature, np.float32) / 255. 
x_test_data = x_test_data.reshape(-1,img_rows,img_cols,3)
print(x_test_data.shape)

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense,Dropout
from keras.layers import Convolution2D 
from keras.layers import MaxPooling2D 
from keras.layers import Flatten

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Convolution2D(32,(3,3),input_shape = (img_cols,img_rows,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Convolution2D(64,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))


classifier.add(Flatten())
classifier.add(Dropout(0.2))
#classifier.add(Dense(units=32,activation='relu'))
#classifier.add(Dense(units=64,activation='relu'))
#classifier.add(Dense(units=120,activation='relu'))
classifier.add(Dense(units = 120, activation = 'softmax'))

In [ ]:
classifier.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"]) 
classifier.summary()

In [ ]:
history = classifier.fit(x_train,y_train,
         batch_size = 128,
         epochs = 20,
         verbose = 2,
         validation_data = (x_val,y_val) 
         )

In [ ]:
results = classifier.predict(x_test_data)
prediction = pd.DataFrame(results)
prediction.columns = encoded_labels.columns.values
prediction.insert(0, 'id', submission['id'])
submission = prediction
submission.to_csv('new_submission.csv', index=False)